In [ ]:
import time
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import string
import random
import scipy
import sklearn.metrics as metrics

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
%matplotlib inline

from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sys import stdout
from operator import itemgetter

N = 1000
alphabet = string.ascii_lowercase + '#' + '.'
D = len(alphabet)
z = np.ones(N)

RI_pres = np.random.rand(D, N)
RI_pres = np.where(RI_pres>0.5, 1, -1)

RI_past = np.random.rand(D, N)
RI_past = np.where(RI_past>0.5, 1, -1)
    
# data augmentation:
def random_noise(X):
    noise = np.random.normal(0, .01, X.shape)
    return X + noise

def read_csv(filepath):
    rows = []
    with open(filepath, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            rows.append(row)
    return rows

def ngram_encode(ngram_str, letter_vecs, window=3):
    vec = np.zeros(letter_vecs.shape[1])
    
    full_str = '#' + ngram_str + '.'
    
    
    for il, l in enumerate(full_str[:-(window-1)]):
        trivec = letter_vecs[alphabet.find(full_str[il]), :]
        for c3 in range(1, window):
            trivec = trivec * np.roll(letter_vecs[alphabet.find(full_str[il+c3]), :], c3)
            
        vec += trivec
    return vec

def ngram_encode_cl(ngram_str, letter_vecs, window=3):
    vec = np.zeros(letter_vecs.shape[1])
    
    full_str = '#' + ngram_str + '.'
    
    for il, l in enumerate(full_str[:-(window-1)]):
        trivec = letter_vecs[alphabet.find(full_str[il]), :]
        for c3 in range(1, window):
            trivec = trivec * np.roll(letter_vecs[alphabet.find(full_str[il+c3]), :], c3)
            
        vec += trivec
    return 2* (vec + 0.1*(np.random.rand(letter_vecs.shape[1])-0.5) < 0) - 1

def shuffle(Xtrain, ytrain):
    N_s, N_f = Xtrain.shape
    N_t = 2*N_s//3
    N_v = N_s - N_t
    training_indices = np.random.choice(xrange(N_s), N_t, replace = False)
    validation_indices = np.array([i for i in xrange(N_s) if i not in training_indices])

    t_data = Xtrain[training_indices]
    v_data = Xtrain[validation_indices]
    t_y = ytrain[training_indices]
    v_y = ytrain[validation_indices]
    return t_data, v_data, t_y, v_y
    
def load_irreg():
    irreg = read_csv("wickle_train/irregular_verbs.csv")
    irreg_past_tense, irreg_words = [], []
    for row in irreg:
        irreg_words.append(row[0])
        irreg_past_tense.append(row[1])
    irreg_present = np.zeros((len(irreg_words), N))
    irreg_past = np.zeros((len(irreg_words), N))
    for i in range(irreg_present.shape[0]):
        irreg_present[i] = ngram_encode_cl(irreg_words[i], RI_pres)
        irreg_past[i] = ngram_encode_cl(irreg_past_tense[i], RI_past)
    return shuffle(irreg_present, irreg_past)

def load_reg():
    reg_present = read_csv("wickle_train/present_reg.csv")
    reg_past = read_csv("wickle_train/past_reg.csv")
    
    pres_enc = []
    past_enc = []
    num_words = len(reg_past)
    N_s = num_words
    N_t = 2*N_s//3
    N_v = N_s - N_t
    training_indices = np.random.choice(xrange(N_s), N_t, replace = False)
    validation_indices = np.array([i for i in xrange(N_s) if i not in training_indices])
    enct, encv = [], []
    # introducing present verbs in descending occurrence
    for i in training_indices:
        enct.append([int(reg_present[i][1]), reg_present[i][0], reg_past[i][0]])
    for i in validation_indices:
        encv.append([int(reg_present[i][1]), reg_present[i][0], reg_past[i][0]])
    enct = sorted(enct, reverse=True)
    encv = sorted(encv, reverse=True)
    t_data, t_y, t_ylabels = np.zeros((len(training_indices), N)), np.zeros((len(training_indices), N)), []
    v_data, v_y, v_ylabels = np.zeros((len(validation_indices), N)), np.zeros((len(validation_indices), N)), []

    for i in range(N_t):
        t_data[i] = ngram_encode_cl(enct[i][1], RI_pres)
        t_y[i] = ngram_encode_cl(enct[i][2], RI_past)
        t_ylabels.append(enct[i][2])
    
    for i in range(N_v):
        v_data[i] = ngram_encode_cl(encv[i][1], RI_pres)
        v_y[i] = ngram_encode_cl(encv[i][2], RI_past)
        v_ylabels.append(encv[i][2])
    print (t_data.shape, t_y.shape, v_data.shape, v_y.shape)
    return t_data, t_y, t_ylabels, v_data, v_y, v_ylabels
    

#load_irreg()
#load_reg()

In [ ]:
def stackones(A, axis=0):
    assert axis in [0, 1]
    if axis == 1:
        return np.hstack((A, np.ones((A.shape[0], 1))))
    return np.vstack((A, np.ones((1, A.shape[1]))))

def softmax(z):
    #Apply softmax activation function
    #use exp-normalize trick
    e_z = np.exp(z - np.max(z))
    return e_z / e_z.sum()

def relu(z):
    return np.maximum(0,z)

def relu_prime(z):
    return 1. * (z > 0)
    #return np.log((1. * (z > 0))+1e-10)


class Neural_Network(object):
    def __init__(self, X, X_v, y, y_v, l, l_v, epsilon=1e-3, epochs=10, reg=0.1, log_lower_bound=.0001, W=None, V=None):        
        #Define Hyperparameters
        self.n_in = X.shape[1]
        self.n_out = y.shape[1]
        self.n_hid = 200
        self.sigma = .01
        #Regularization Parameter:
        self.reg = 0
        #learning rate
        self.epsilon = epsilon # or is it 1e-2 .01
        # learning rate decay. usually .5 or .9
        self.decay = .8
        self.log_lower_bound = log_lower_bound
        self.t = time.time()
        self.training_sim = []
        self.validation_sim = []
        self.random_sim = []
        self.running_time = []
        self.training_losses = []
        self.validation_losses = []
        self.epochs = epochs
        #Define Data
        self.X = X
        self.X_v = X_v
        self.y = y
        self.y_v = y_v 
        self.l = l
        self.l_v = l_v
        #Weights (parameters)
        # to initialize as gaussian with mean = 0, variance < 1.
        if W == None:
            self.W = np.random.normal(0, self.sigma, (self.n_out, self.n_hid + 1))
        else:
            self.W = W
        if V == None:
            self.V = np.random.normal(0, self.sigma, (self.n_hid, self.n_in + 1))
        else:
            self.V = V
    
    def forward(self, X):
        #print(X.shape)
        self.inp = stackones(X, axis=1)
        self.z1 = self.inp.dot(self.V.T)
        self.a1 = relu(self.z1) # (1, 200)
        self.hid = stackones(self.a1, axis=1)
        self.z2 = self.hid.dot(self.W.T)
        self.out = softmax(self.z2)
        return self.out

    def predict(self, X):
        inp = stackones(X, axis=1)
        z1 = inp.dot(self.V.T)
        a1 = relu(z1)
        hid = stackones(a1, axis=1)
        z2 = hid.dot(self.W.T)
        out = softmax(z2)
        return out
        #yHat = np.argmax(out, 1)
        #return yHat

    def cross_entropy_loss(self, X, y):
        y = np.matrix(y)
        self.j = -y.dot(np.log(self.out.T+1e-10))
        return self.j
    
    def loss_prime(self, X, y):
        #Compute derivative with respect to W and V for a given X and y:
        # (1, 200)
        self.h = relu(self.inp.dot(self.V.T)) # hidden layer
        self.h = stackones(self.h, axis=1)
        
        self.deltaW = self.out-y # the delta with respect to W
        # n_out by n_hid+1
        print self.deltaW.shape
        self.dJdW = self.deltaW.reshape((N,1)).dot(self.h)
        self.dJdV = 0
        
        X = np.matrix(X) 
        X = stackones(X, axis=1) # (1, 785)
        self.dJdh = self.W.T.dot(self.deltaW.T)
        self.deltaV = np.multiply(self.dJdh[:self.dJdh.shape[0]-1,:], relu_prime(self.V.dot(X.T))) # (200,1)
        self.dJdV = self.deltaV.dot(X) 
        
        return self.dJdW, self.dJdV

    def backprop(self, V, W, X, y, epsilon):
        self.loss_prime(X, y)
        return V-epsilon*self.dJdV, W-epsilon*self.dJdW
    
    # train with stochastic gradient descent
    def train(self):
        m, s = divmod(time.time() - self.t, 60)
        self.running_time.append(m)

        print("Trial with epsilon={}. Time elapsed: {} minutes, {} seconds.".format(self.epsilon, m, s))
        #print self.X.shape[0]
        for i in range(self.epochs):
            indices = np.random.permutation(xrange(self.X.shape[0]))
            
            for j in range(self.X.shape[0]):
            #for j in range(100):
                if j%2 == 0:
                    y = self.y[indices[j], :]
                    X = self.X[indices[j], :].reshape((1, self.X.shape[1]))
                    out = self.forward(X)
                    self.V, self.W = self.backprop(self.V, self.W, X, y, self.epsilon)
                    self.j = self.cross_entropy_loss(X,y)
                    stdout.write("\r[ITERATION] {} J = {}".format(j + 1, self.j))
                    stdout.flush()

                    #if j%100 == 0 or j == self.X.shape[0]-1:
                    
                    self.training_losses.append(np.sum(self.cross_entropy_loss(self.X, self.y)))
                    self.validation_losses.append(np.sum(self.cross_entropy_loss(self.X_v, self.y_v)))
                    yHat = self.predict(self.X)
                    yHat_v = self.predict(self.X_v)
                    self.training_sim.append(np.mean(self.y.T.dot(yHat)))
                    self.validation_sim.append(np.mean(self.y_v.T.dot(yHat_v)))
                    self.random_sim.append(np.mean(self.y_v.T.dot(np.random.rand(self.X_v.shape[0], self.X_v.shape[1]))))
            if i%2 == 0:
                self.epsilon = self.decay*self.epsilon
                
        print "D O N E"
        return self

def linear_sweep():
    # linear sweep of all the epsilons
    epsilon = 1e-2
    for i in range(0,1000):
        print ("EPSILON: {}".format(epsilon-i*1e-5))
        nn = Neural_Network(X_train, X_validate, y_train, y_validate, labels_train, labels_validate, epsilon-i*1e-5)
        nn.train() 

def writeup(epsilon=.00999,epochs=1):
    X_train, y_train, labels_train, X_validate, y_validate, labels_validate = load_reg()
    nn = Neural_Network(X_train, X_validate, y_train, y_validate, labels_train, labels_validate, epsilon, epochs)
    nn.train()
    plt.figure()
    x = np.linspace(0, len(nn.training_losses), len(nn.training_losses))
    plt.plot(x, np.array(nn.training_losses), label='training losses')
    plt.plot(x, np.array(nn.validation_losses), label='validation losses')
    plt.legend(loc='best')
    plt.title('Loss vs iterations')
    plt.xlabel('Number of iterations in 10,000s')
    plt.ylabel('Total loss')
    plt.show()
    
    plt.figure()
    x = np.linspace(0, len(nn.training_sim), len(nn.training_sim))
    plt.plot(x, np.array(nn.training_sim), label='training similarity')
    plt.plot(x, np.array(nn.validation_sim), label='validation similarity')
    plt.plot(x, np.array(nn.random_sim), label='random similarity')
    plt.legend(loc='best')
    plt.title('Similarity vs iterations')
    plt.xlabel('Number of iterations')
    plt.ylabel('Average Similarity')
    plt.show()
    

writeup()  

((1074, 1000), (1074, 1000), (537, 1000), (537, 1000))
Trial with epsilon=0.00999. Time elapsed: 0.0 minutes, 0.0225689411163 seconds.
(1, 1000)
[ITERATION] 1 J = [[ 262.86725518]](1, 1000)
[ITERATION] 3 J = [[ 90.70319081]](1, 1000)
[ITERATION] 5 J = [[-268.84730848]](1, 1000)
[ITERATION] 7 J = [[ 222.77637253]](1, 1000)
[ITERATION] 9 J = [[ 241.42213427]](1, 1000)
[ITERATION] 11 J = [[-187.75256342]](1, 1000)
[ITERATION] 13 J = [[-138.42634844]](1, 1000)
[ITERATION] 15 J = [[ 7.00783902]](1, 1000)
[ITERATION] 17 J = [[ 300.61546807]](1, 1000)
[ITERATION] 19 J = [[ 403.02791082]](1, 1000)
[ITERATION] 21 J = [[ 233.58050023]](1, 1000)
[ITERATION] 23 J = [[ 219.68059912]](1, 1000)
[ITERATION] 25 J = [[-360.40162544]](1, 1000)
[ITERATION] 27 J = [[-777.25592819]](1, 1000)
[ITERATION] 29 J = [[-63.72888095]](1, 1000)
[ITERATION] 31 J = [[-168.36980364]](1, 1000)
[ITERATION] 33 J = [[-787.45202537]](1, 1000)
[ITERATION] 35 J = [[-1468.96187694]](1, 1000)
[ITERATION] 37 J = [[ 2132.79550253